In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc = SparkContext()
ssc = StreamingContext(sc, 5)

In [3]:
lines = ssc.socketTextStream("localhost", 9999)

In [4]:
type(lines)

pyspark.streaming.dstream.DStream

# Echo streaming

### Inizializzo lo stream da terminale con l'istruzione nc -lk 9999

In [5]:
lines.pprint()

In [6]:
# ssc.start()
# ssc.awaitTermination()

# Words streaming

In [7]:
words = lines.flatMap(lambda text: text.split())
words.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-03-24 07:36:50
-------------------------------------------

-------------------------------------------
Time: 2020-03-24 07:36:50
-------------------------------------------

-------------------------------------------
Time: 2020-03-24 07:36:55
-------------------------------------------

-------------------------------------------
Time: 2020-03-24 07:36:55
-------------------------------------------

-------------------------------------------
Time: 2020-03-24 07:37:00
-------------------------------------------
Ciao a tutti come va?

-------------------------------------------
Time: 2020-03-24 07:37:00
-------------------------------------------
Ciao
a
tutti
come
va?

